In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Faz o cruzamento de deputados e doações - para saber deputados que podem estar nos autos de infração do ICMBio ou doadores que estão na lista de autos
# Dados obtidos pela reportagem da RB

In [1]:
import dbf
import pandas as pd
from dbfread import DBF
from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [3]:
dbf = Dbf5('originais/Autos_Infracao_Eletronicos_ICMBio_Atualizado_12072021.dbf', codec='ISO8859-1')

In [4]:
autos_icmbio = dbf.to_dataframe()

In [5]:
autos_icmbio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   vw_num_aut  2038 non-null   int64  
 1   numero_ai   2038 non-null   object 
 2   status      2038 non-null   object 
 3   tipo        2038 non-null   object 
 4   embargo     2038 non-null   object 
 5   autuado     2038 non-null   object 
 6   cpf_cnpj    1882 non-null   object 
 7   valor_mult  2014 non-null   object 
 8   desc_infra  2011 non-null   object 
 9   tipo_inf    2038 non-null   object 
 10  data        2038 non-null   object 
 11  nome_uc     2038 non-null   object 
 12  cnuc        2038 non-null   object 
 13  municipio   2038 non-null   object 
 14  cod_muni    2038 non-null   object 
 15  uf          2038 non-null   object 
 16  operacao    2021 non-null   object 
 17  orgao_emis  2038 non-null   object 
 18  agente      753 non-null    object 
 19  enquadrame  2038 non-null  

In [6]:
autos_icmbio[['cpf_cnpj', 'autuado']].sample(5)

,cpf_cnpj,autuado
998,00398966923,WILLIAM MARTINS DOS SANTOS
982,62574299000,GILSO UNCINI
595,NaN,ClÃ¡udio da Rosa
1016,84571004249,Jorge Souza Cruz
1240,02360400304,Jose Valter Vasconcelos


In [7]:
autos_icmbio = autos_icmbio[pd.notnull(autos_icmbio['cpf_cnpj'])].copy()

In [8]:
def limpeza1(linha):
    documento = str(linha["cpf_cnpj"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)

In [9]:
autos_icmbio["cpf_cnpj"] = autos_icmbio.apply(limpeza1, axis=1)

In [10]:
autos_icmbio[['cpf_cnpj', 'autuado']].sample(5)

,cpf_cnpj,autuado
818,60111151287,Tercilia Pereira da Silva
1322,05715452201,Raimundo Nonato Pereira de Sousa JÃºnior
1099,01572381302,Alessandro Ferreira dos Santos
1072,93607687587,Doraci Antonio Fernandes
1378,99529050097,Mauricio Lhulier de Souza


In [11]:
# Preenche zeros a esquerda nos documentos
def compara1(linha):
    documento = str(linha["cpf_cnpj"])
    
    if len(documento) < 14 and len(documento) >= 12:
        conta = 14 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 14:
        alterado = documento
    elif len(documento) < 11:
        conta = 11 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 11:
        alterado = documento
    
    return alterado

In [12]:
autos_icmbio["cpf_cnpj"] = autos_icmbio.apply(compara1, axis=1)

In [13]:
def limpeza2(linha):
    documento = str(linha["cpf_cnpj"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    
autos_icmbio['cnpj_raiz'] = autos_icmbio.apply(limpeza2, axis=1)

In [14]:
autos_icmbio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1882 entries, 0 to 2037
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   vw_num_aut  1882 non-null   int64  
 1   numero_ai   1882 non-null   object 
 2   status      1882 non-null   object 
 3   tipo        1882 non-null   object 
 4   embargo     1882 non-null   object 
 5   autuado     1882 non-null   object 
 6   cpf_cnpj    1882 non-null   object 
 7   valor_mult  1862 non-null   object 
 8   desc_infra  1855 non-null   object 
 9   tipo_inf    1882 non-null   object 
 10  data        1882 non-null   object 
 11  nome_uc     1882 non-null   object 
 12  cnuc        1882 non-null   object 
 13  municipio   1882 non-null   object 
 14  cod_muni    1882 non-null   object 
 15  uf          1882 non-null   object 
 16  operacao    1865 non-null   object 
 17  orgao_emis  1882 non-null   object 
 18  agente      718 non-null    object 
 19  enquadrame  1882 non-null  

In [15]:
icmbio_copia = autos_icmbio.copy()
icmbio_copia.columns

Index(['vw_num_aut', 'numero_ai', 'status', 'tipo', 'embargo', 'autuado',
       'cpf_cnpj', 'valor_mult', 'desc_infra', 'tipo_inf', 'data', 'nome_uc',
       'cnuc', 'municipio', 'cod_muni', 'uf', 'operacao', 'orgao_emis',
       'agente', 'enquadrame', 'cod_acao', 'termo_emb', 'obs', 'erro_auto',
       'nr_process', 'julgamento', 'cnpj_raiz'],
      dtype='object')

In [16]:
icmbio_copia

,vw_num_aut,numero_ai,status,tipo,embargo,autuado,cpf_cnpj,valor_mult,desc_infra,tipo_inf,...,orgao_emis,agente,enquadrame,cod_acao,termo_emb,obs,erro_auto,nr_process,julgamento,cnpj_raiz
0,1,009Z5YHU,Impresso,Multa Simples,F,Luiz Oscar Marques Ferreira,30989388115,500,NaN,Unidade de ConservaÃ§Ã£o,...,ICMBio,Marcela de Marins,90 Decreto 6514; 70 1ÃÂº Lei 9605; 72 Lei 960...,G5VODTH,NaN,NaN,NaN,02125000946202047,NaN,documento_cpf
1,2,02XTX4FP,Impresso,Multa Simples,F,Francisco Alcenir Rodrigues de Souza,31753302000164,1000,"realizar serviÃ§o comercial "" passeio de quadr...",Unidade de ConservaÃ§Ã£o,...,ICMBio,NaN,90 Decreto 6514; 70 1ÃÂº Lei 9605; 72 Lei 960...,887OF6C,NaN,NaN,NaN,02123000251202166,NaN,31753302
2,3,036GW0EM,Impresso,Multa Simples,F,CIDADE JARDIM COMERCIO DE MADEIRAS LTDA,22327400000149,500,NaN,Unidade de ConservaÃ§Ã£o,...,ICMBio,NaN,90 Decreto 6514; 70 1ÃÂº Lei 9605; 72 Lei 960...,BH0IA1H,NaN,NaN,NaN,02126004316201906,NaN,22327400
3,4,03BPDFMZ,Impresso,Multa Simples,T,Luiz Felipe Grava do Val Nascimento,15813798897,590000,"Destruir 58,7062 hectares de floresta nativa, ...",Unidade de ConservaÃ§Ã£o,...,ICMBio,NaN,50 Decreto 6514; 70 1ÃÂº Lei 9605; 72 Lei 960...,CG53IYP,TOPRRT4Z,NaN,NaN,02119000580202095,NaN,documento_cpf
4,5,03LF39RR,Impresso,Multa Simples,T,Vilson Predolim,04701852872,10200,Causar dano a Ãrea de ProteÃ§Ã£o Ambiental de...,Unidade de ConservaÃ§Ã£o,...,ICMBio,NaN,91 Decreto 6514; 70 1ÃÂº Lei 9605; 72 Lei 960...,OH0BZAN,2TUPM4SA,NaN,NaN,02126000717202101,NaN,documento_cpf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2034,ZYCDFDO5,Impresso,Multa Simples,F,Osmar Leandro da Silva,90390148253,87500,"destruir 8,75 hectares de floresta nativa sem ...",Flora,...,ICMBio,NaN,50 Decreto 6514; 70 1ÃÂº Lei 9605; 72 Lei 960...,PNC26VY,NaN,NaN,NaN,02119001499202022,NaN,documento_cpf
2034,2035,ZZ4XL2OM,Impresso,Multa Simples,F,Clodomir Ceolatto,38384698104,1400,"Pescar em local proibido, no entorno da EstaÃ§...",Pesca,...,ICMBio,NaN,35 Decreto 6514; 42 Decreto 6514; 93 Decreto 6...,DP6Q4EQ,NaN,NaN,NaN,02129000426202002,NaN,documento_cpf
2035,2036,ZZJRMCBD,Impresso,Multa Simples,T,Inacio Liber da Cruz,52712826272,4836000,"Destruir 402,3 hectares de floresta nativa per...",Flora,...,ICMBio,NaN,49 Decreto 6514; 93 Decreto 6514; 70 1ÃÂº Lei...,I63KD3W,"1XC8K8HJ, 1XC8K8HJ",NaN,NaN,02127001591202093,NaN,documento_cpf
2036,2037,ZZK7POZY,Impresso,Multa Simples,T,Luiz OtÃ¡vio Affonso Christo,00313391742,21200,Causar dano a APA CairuÃ§u por construir deck ...,Unidade de ConservaÃ§Ã£o,...,ICMBio,NaN,91 Decreto 6514; 70 1ÃÂº Lei 9605; 72 Lei 960...,2H1ZFWK,CR84JVV5,NaN,NaN,02126002790202029,NaN,documento_cpf


In [17]:
empresas_encontradas = autos_icmbio[(autos_icmbio['cpf_cnpj'] == '31583660615')]
empresas_encontradas

,vw_num_aut,numero_ai,status,tipo,embargo,autuado,cpf_cnpj,valor_mult,desc_infra,tipo_inf,...,orgao_emis,agente,enquadrame,cod_acao,termo_emb,obs,erro_auto,nr_process,julgamento,cnpj_raiz


In [20]:
empresas_encontradas = autos_icmbio[(autos_icmbio['cpf_cnpj'] == '70335591787')]
empresas_encontradas

,vw_num_aut,numero_ai,status,tipo,embargo,autuado,cpf_cnpj,valor_mult,desc_infra,tipo_inf,...,orgao_emis,agente,enquadrame,cod_acao,termo_emb,obs,erro_auto,nr_process,julgamento,cnpj_raiz


In [21]:
empresas_encontradas = autos_icmbio[(autos_icmbio['cpf_cnpj'] == '04856647040')]
empresas_encontradas

,vw_num_aut,numero_ai,status,tipo,embargo,autuado,cpf_cnpj,valor_mult,desc_infra,tipo_inf,...,orgao_emis,agente,enquadrame,cod_acao,termo_emb,obs,erro_auto,nr_process,julgamento,cnpj_raiz


In [19]:
new = autos_icmbio["cnpj_raiz"].isin(["01463126", "02657326", "03167408", "04057706", "05904108", "07353227", "10242314", "12631778", "13134686", "14019929", "17331307", "21049762", "21129267", "22158240", "25555723", "26172122", "30202385", "32161939"])
autos_icmbio[new]  

,vw_num_aut,numero_ai,status,tipo,embargo,autuado,cpf_cnpj,valor_mult,desc_infra,tipo_inf,...,orgao_emis,agente,enquadrame,cod_acao,termo_emb,obs,erro_auto,nr_process,julgamento,cnpj_raiz


In [17]:
icmbio_copia.rename(columns = {'desc_infra':'tipo_infracao'},inplace = True)
icmbio_copia.rename(columns = {'data':'data_fiscalizacao'},inplace = True)
icmbio_copia.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
icmbio_copia.rename(columns = {'autuado':'entidade_nome'},inplace = True)
icmbio_copia.rename(columns = {'uf':'estado_id'},inplace = True)
icmbio_copia.rename(columns = {'valor_mult':'valor_multa'},inplace = True)

In [18]:
icmbio_copia = icmbio_copia[['municipio', 'tipo_infracao', 'data_fiscalizacao', 'entidade_id', 'entidade_nome', 'estado_id', 'valor_multa']]

In [19]:
icmbio_copia.to_csv('resultados/autos_icmbio/lista_entidades_autos_icmbio.csv',index=False)

In [20]:
icmbio_copia.to_excel('resultados/autos_icmbio/lista_entidades_autos_icmbio_todas_colunas.xlsx',sheet_name='Sheet1',index=False)

In [29]:
#ibama_menor = ibama[['cpf_cnpj', 'Nome ou Razão Social', 'cnpj_raiz']]

In [19]:
icmbio_pf = autos_icmbio[(autos_icmbio['cnpj_raiz'] == 'documento_cpf')]
icmbio_pj = autos_icmbio[(autos_icmbio['cnpj_raiz'] != 'documento_cpf')]

In [20]:
icmbio_pf.shape

(1799, 27)

In [21]:
icmbio_pj.shape

(83, 27)

In [22]:
busca = autos_icmbio[(autos_icmbio['cpf_cnpj'] == '44444444444')]
busca

,vw_num_aut,numero_ai,status,tipo,embargo,autuado,cpf_cnpj,valor_mult,desc_infra,tipo_inf,...,orgao_emis,agente,enquadrame,cod_acao,termo_emb,obs,erro_auto,nr_process,julgamento,cnpj_raiz


### Doações

In [22]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
doacoes_2018 = pd.read_csv("resultados/doacoes/doacoes_geral_politicos_2018.csv", **kwargs)

In [23]:
def limpeza3(linha):
    documento = str(linha["NR_CPF_CNPJ_DOADOR"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] = doacoes_2018.apply(limpeza3, axis=1)

In [24]:
doacoes_2018_pf = doacoes_2018[(doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] == 'documento_cpf')]
doacoes_2018_pj = doacoes_2018[(doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] != 'documento_cpf')]

In [25]:
doacoes_pj_icmbio = pd.merge(doacoes_2018, icmbio_pj.drop_duplicates('cnpj_raiz'), left_on='NR_CPF_CNPJ_DOADOR_raiz', right_on='cnpj_raiz', how = 'left')

In [26]:
conta = doacoes_pj_icmbio[pd.notnull(doacoes_pj_icmbio['cpf_cnpj'])]
conta.shape

(0, 43)

In [28]:
# Pessoa física

In [27]:
doacoes_pf_icmbio = pd.merge(doacoes_2018, icmbio_pf.drop_duplicates('cpf_cnpj'), left_on='NR_CPF_CNPJ_DOADOR', right_on='cpf_cnpj', how = 'left')

In [28]:
conta = doacoes_pf_icmbio[pd.notnull(doacoes_pf_icmbio['cpf_cnpj'])].copy()
conta.shape

(3, 43)

In [52]:
# Com todas colunas

In [53]:
conta.to_excel('resultados/autos_icmbio/doacoes_vindas_pessoas_com_autos_icmbio_2018.xlsx',sheet_name='Sheet1',index=False)

In [54]:
# Versão para BD

In [29]:
conta['doacoes_tipo'] = 'direta'

In [30]:
conta.columns

Index(['DT_RECEITA', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'NM_CANDIDATO',
       'DS_FONTE_RECEITA', 'DS_NATUREZA_RECEITA', 'NR_CPF_CNPJ_DOADOR',
       'NM_DOADOR_RFB', 'DS_ESFERA_PARTIDARIA_DOADOR', 'NM_PARTIDO_DOADOR',
       'SQ_CANDIDATO_DOADOR', 'DS_CARGO_CANDIDATO_DOADOR', 'SG_PARTIDO_DOADOR',
       'DS_RECEITA', 'VR_RECEITA', 'NR_CPF_CNPJ_DOADOR_raiz', 'vw_num_aut',
       'numero_ai', 'status', 'tipo', 'embargo', 'autuado', 'cpf_cnpj',
       'valor_mult', 'desc_infra', 'tipo_inf', 'data', 'nome_uc', 'cnuc',
       'municipio', 'cod_muni', 'uf', 'operacao', 'orgao_emis', 'agente',
       'enquadrame', 'cod_acao', 'termo_emb', 'obs', 'erro_auto', 'nr_process',
       'julgamento', 'cnpj_raiz', 'doacoes_tipo'],
      dtype='object')

In [31]:
conta.rename(columns = {'VR_RECEITA':'valor'},inplace = True)
conta.rename(columns = {'NR_CPF_CANDIDATO':'politico_id'},inplace = True)
conta.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
conta.rename(columns = {'DS_FONTE_RECEITA':'origem_do_dinheiro_no_tse'},inplace = True)
conta.rename(columns = {'DT_RECEITA':'data_receita'},inplace = True)
conta.rename(columns = {'NM_DOADOR_RFB':'entidade_nome'},inplace = True)

In [32]:
conta = conta[['valor', 'politico_id', 'entidade_id', 'entidade_nome', 'doacoes_tipo', 'origem_do_dinheiro_no_tse', 'data_receita']]

In [33]:
conta.columns

Index(['valor', 'politico_id', 'entidade_id', 'entidade_nome', 'doacoes_tipo',
       'origem_do_dinheiro_no_tse', 'data_receita'],
      dtype='object')

In [34]:
conta.rename(columns = {'data_receita':'data'},inplace = True)

In [35]:
conta.shape

(3, 7)

In [38]:
conta

,valor,politico_id,entidade_id,entidade_nome,doacoes_tipo,origem_do_dinheiro_no_tse,data
14306,"2000,00",09911967751,76623670700,MARCELO PINTO DUARTE BARBARA,direta,Outros Recursos,03/10/2018
14502,"2000,00",08868429721,76623670700,MARCELO PINTO DUARTE BARBARA,direta,Outros Recursos,03/10/2018
20701,"10000,00",34806872881,76623670700,MARCELO PINTO DUARTE BARBARA,direta,Outros Recursos,27/08/2018


In [39]:
conta.to_csv('resultados/autos_icmbio/doacoes_vindas_pessoas_com_autos_icmbio_2018.csv',index=False)

### Doações indiretas

In [36]:
doacoes_indiretas_2018 = pd.read_excel('resultados/doacoes/doacoes_todas_indiretas_politicos_2018.xlsx', sheet_name = 'Sheet1', dtype = 'str')

In [37]:
doacoes_indiretas_2018.shape

(2372, 68)

In [38]:
doacoes_indiretas_2018.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'SG_UE', 'NM_UE', 'NR_CNPJ_PRESTADOR_CONTA', 'CD_CARGO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO', 'NR_CPF_VICE_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'CD_FONTE_RECEITA', 'DS_FONTE_RECEITA',
       'CD_ORIGEM_RECEITA', 'DS_ORIGEM_RECEITA', 'CD_NATUREZA_RECEITA',
       'DS_NATUREZA_RECEITA', 'CD_ESPECIE_RECEITA', 'DS_ESPECIE_RECEITA',
       'CD_CNAE_DOADOR', 'DS_CNAE_DOADOR', 'NR_CPF_CNPJ_DOADOR', 'NM_DOADOR',
       'NM_DOADOR_RFB', 'CD_ESFERA_PARTIDARIA_DOADOR',
       'DS_ESFERA_PARTIDARIA_DOADOR', 'SG_UF_DOADOR', 'CD_MUNICIPIO_DOADOR',
       'NM_MUNICIPIO_DOADOR', 'SQ_CANDIDATO_DOADOR', 'NR_CANDIDATO_DOADOR',
       'CD_CARGO_CANDIDATO_DOADOR', 'DS_CARGO_CANDIDATO_DOADOR

In [39]:
doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] = doacoes_indiretas_2018.apply(limpeza3, axis=1)

In [40]:
doacoes_indiretas_2018_pf = doacoes_indiretas_2018[(doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] == 'documento_cpf')]
doacoes_indiretas_2018_pj = doacoes_indiretas_2018[(doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] != 'documento_cpf')]

In [41]:
doacoes_indiretas_2018_pf.shape

(0, 69)

In [42]:
doacoes_indiretas_2018_pj.shape

(2372, 69)

In [43]:
doacoes_pf_icmbio_indireta = pd.merge(doacoes_indiretas_2018_pj, icmbio_pf.drop_duplicates('cpf_cnpj'), left_on='cpf_doacoaindireta', right_on='cpf_cnpj', how = 'left')

In [44]:
conta = doacoes_pf_icmbio_indireta[pd.notnull(doacoes_pf_icmbio_indireta['cpf_cnpj'])].copy()
conta.shape

(0, 96)

In [58]:
#conta.to_excel('resultados/embargos_ibama/doacoes_indiretas_vindas_pessoas_com_embargos_ibama_2018.xlsx',sheet_name='Sheet1',index=False)

In [59]:
#conta['doacoes_tipo'] = 'indireta'

In [61]:
#conta.columns

In [61]:
#conta.rename(columns = {'VR_RECEITA':'valor'},inplace = True)
#conta.rename(columns = {'NR_CPF_CANDIDATO':'politico_id'},inplace = True)
#conta.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
#conta.rename(columns = {'DS_FONTE_RECEITA':'origem_do_dinheiro_no_tse'},inplace = True)
#conta.rename(columns = {'DT_RECEITA':'data'},inplace = True)
#conta.rename(columns = {'SQ_CANDIDATO_DOADOR':'sequencialdoador_id'},inplace = True)

In [62]:
#conta = conta[['valor', 'politico_id', 'entidade_id', 'sequencialdoador_id', 'doacoes_tipo', 'origem_do_dinheiro_no_tse', 'data']]

In [63]:
#conta.columns

In [62]:
#conta.head()

In [64]:
#conta.shape

In [66]:
#conta.to_csv('resultados/embargos_ibama/doacoes_indiretas_vindas_pessoas_com_embargos_ibama_2018.csv',index=False)

### Doações originárias

In [45]:
doacoes_originarias_2018 = pd.read_excel('resultados/doacoes/doacoes_originarias_politicos_2018.xlsx', sheet_name = 'Sheet1', dtype = 'str')

In [46]:
doacoes_originarias_2018.shape

(11060, 25)

In [47]:
doacoes_originarias_2018.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'NR_CPF_CNPJ_DOADOR_ORIGINARIO', 'NM_DOADOR_ORIGINARIO',
       'NM_DOADOR_ORIGINARIO_RFB', 'TP_DOADOR_ORIGINARIO',
       'CD_CNAE_DOADOR_ORIGINARIO', 'DS_CNAE_DOADOR_ORIGINARIO', 'SQ_RECEITA',
       'DT_RECEITA', 'DS_RECEITA', 'VR_RECEITA', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO'],
      dtype='object')

In [48]:
def limpeza4(linha):
    documento = str(linha["NR_CPF_CNPJ_DOADOR_ORIGINARIO"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    
doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] = doacoes_originarias_2018.apply(limpeza4, axis=1)

In [49]:
doacoes_originarias_2018_pf = doacoes_originarias_2018[(doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] == 'documento_cpf')]
doacoes_originarias_2018_pj = doacoes_originarias_2018[(doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] != 'documento_cpf')]

In [50]:
doacoes_originarias_2018_pf.shape

(11060, 26)

In [51]:
doacoes_originarias_2018_pj.shape

(0, 26)

In [53]:
doacoes_pf_icmbio_originaria = pd.merge(doacoes_originarias_2018_pf, icmbio_pf.drop_duplicates('cpf_cnpj'), left_on='NR_CPF_CNPJ_DOADOR_ORIGINARIO', right_on='cpf_cnpj', how = 'left')

In [54]:
conta = doacoes_pf_icmbio_originaria[pd.notnull(doacoes_pf_icmbio_originaria['cpf_cnpj'])].copy()
conta.shape

(0, 53)

In [90]:
#conta.to_excel('resultados/embargos_ibama/doacoes_originarias_vindas_pessoas_com_embargos_ibama_2018.xlsx',sheet_name='Sheet1',index=False)

In [91]:
#conta['doacoes_tipo'] = 'originaria'

In [74]:
#conta.columns

In [93]:
#conta.rename(columns = {'VR_RECEITA':'valor'},inplace = True)
#conta.rename(columns = {'NR_CPF_CANDIDATO':'politico_id'},inplace = True)
#conta.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
#conta.rename(columns = {'DS_RECEITA':'origem_do_dinheiro_no_tse'},inplace = True)
#conta.rename(columns = {'DT_PRESTACAO_CONTAS':'data'},inplace = True)

In [94]:
#conta = conta[['valor', 'politico_id', 'entidade_id', 'doacoes_tipo', 'origem_do_dinheiro_no_tse', 'data']]

In [75]:
#conta.columns

In [76]:
#conta.head()

In [97]:
#conta.to_csv('resultados/embargos_ibama/doacoes_originarias_vindas_pessoas_com_embargos_ibama_2018.csv',index=False)

### Candidatos 2018

In [56]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
politicos_2018 = pd.read_csv("resultados/politicos_2018_09abr.csv", **kwargs)

In [57]:
politicos_autos_icmbio = pd.merge(icmbio_pf.drop_duplicates('cpf_cnpj'), politicos_2018, left_on='cpf_cnpj', right_on='politico_id', how = 'left')

In [58]:
conta = politicos_autos_icmbio[pd.notnull(politicos_autos_icmbio['politico_id'])].copy()
conta.shape

(0, 36)

In [59]:
#conta

In [60]:
#conta.to_excel('resultados/embargos_ibama/deputados_2018_com_embargos_ibama.xlsx',sheet_name='Sheet1',index=False)

In [175]:
#conta.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
#conta.rename(columns = {'Município Infração':'municipio'},inplace = True)
#conta.rename(columns = {'Infração':'tipo_infracao'},inplace = True)
#conta.rename(columns = {'Data de Inserção na Lista':'data_insercao_lista'},inplace = True)
#conta.rename(columns = {'UF Infração':'estado_id'},inplace = True)
#conta.rename(columns = {'Área (ha)':'tamanho_area'},inplace = True)

In [176]:
#conta = conta[['entidade_id', 'municipio', 'tipo_infracao', 'data_insercao_lista', 'estado_id', 'tamanho_area']]

In [178]:
#conta.to_csv('resultados/embargos_ibama/deputados_2018_com_embargos_ibama.csv',index=False)